## 划分训练集和测试集

### 1.随机抽样

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split( data, test_size=0.2, random_state=42)

test_size: 测试集大小<br/>
random_state: 随机种子

### 2.分层抽样

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data["income_cat"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

n_splits是将训练数据分成train/test对的组数，可根据需要进行设置，默认为10(此处n_splits=1)

## 数据清洗

缺失特征数据的处理:<br/> 
1.剔除缺失该特征的样本<br/>
2.剔除整个特征<br/>
3.用特定值填充缺失的特征数据(0,平均数,中位数等)

In [ ]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

创建Imputer的实例,可以用于向缺失数据填充特定值, 其中中位数只能计算于数值型特征

In [ ]:
imputer.fit(data_num)
X = imputer.transform(data_num)

利用fit()方法对数据进行适应<br/>
利用transform()方法对数据进行转换

In [ ]:
imputer.strategy

获取imputer实例中超参数strategy的值(不带下划线)

In [ ]:
imputer.statistics_

获取imputer实例中参数statistics的值(带下划线) <br/>
使用fit()方法后获得的参数

## 处理文本分类特征

In [ ]:
data_encoded, data_categories = Series.factorize()

将文本分类的Series数据转化为数字分类<br/>
factorize()返回2个对象:<br/>
第一个对象为数字分类的列表<br/>
第二个对象为数字分类对应的标签

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
data_cat_1hot = encoder.fit_transform(data_encoded.reshape(-1,1))

将之前的数字分类,转换为one-hot向量<br/>
默认的输出形式为Scipy的稀疏(sparse)矩阵

In [ ]:
data_cat_1hot.toarray()

通过toarray()方法将稀疏矩阵转化为稠密(dense)矩阵